In [ ]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score

# 데이터 로드
data = pd.read_csv("../../../../monunmon.csv")  # CSV 파일 경로

# 레이블 이진 분류를 위해 변환 (0~94 -> 1, -1 -> 0)
data['Label'] = data['Label'].apply(lambda x: 1 if x >= 0 else 0)

# 변환 결과 확인
print("Label Distribution:")
print(data['Label'].value_counts())

# 특성과 레이블 분리
X = data.drop(columns=['Label'])  # 피처 데이터
y = data['Label']  # 레이블 데이터

# Train, Validation, Test 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# 데이터 정규화 (Logistic Regression은 정규화된 데이터에서 더 잘 작동)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Logistic Regression 모델 정의 및 학습
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train, y_train)

# 검증 데이터 평가
y_val_pred = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_f1 = f1_score(y_val, y_val_pred)

print(f"Validation Accuracy: {val_accuracy:.2f}")
print(f"Validation F1 Score: {val_f1:.2f}")

# 테스트 데이터 평가
y_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)

print(f"Test Accuracy: {test_accuracy:.2f}")
print(f"Test F1 Score: {test_f1:.2f}")

# 분류 보고서 출력
print("\nClassification Report on Test Data:")
print(classification_report(y_test, y_test_pred))


Label Distribution:
Label
1    19000
0     3000
Name: count, dtype: int64
Validation Accuracy: 0.87
Validation F1 Score: 0.93
Test Accuracy: 0.87
Test F1 Score: 0.93

Classification Report on Test Data:
              precision    recall  f1-score   support

           0       0.59      0.04      0.08       450
           1       0.87      1.00      0.93      2850

    accuracy                           0.87      3300
   macro avg       0.73      0.52      0.50      3300
weighted avg       0.83      0.87      0.81      3300



In [ ]:
# 필요한 라이브러리 불러오기
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# 데이터 로드
data = pd.read_csv("../../../../monunmon.csv")  # 데이터셋 경로 입력
data['Label'] = data['Label'].apply(lambda x: 1 if x >= 0 else 0)  # 이진 레이블 변환

# 특성과 레이블 분리
X = data.drop(columns=['Label'])  # 피처 데이터
y = data['Label']  # 레이블 데이터

# Train, Validation, Test 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# 데이터 정규화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

### 방법 1: 클래스 가중치 적용 ###
print("=== Logistic Regression with Class Weights ===")
model_weighted = LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced')
model_weighted.fit(X_train, y_train)

# 테스트 데이터 평가
y_test_pred = model_weighted.predict(X_test)
print("\nClassification Report on Test Data (Class Weights):")
print(classification_report(y_test, y_test_pred))

### 방법 2: SMOTE를 이용한 오버샘플링 ###
print("\n=== Logistic Regression with SMOTE ===")
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print("Resampled Training Set Size (SMOTE):", X_train_smote.shape)

model_smote = LogisticRegression(random_state=42, max_iter=1000)
model_smote.fit(X_train_smote, y_train_smote)

# 테스트 데이터 평가
y_test_pred_smote = model_smote.predict(X_test)
print("\nClassification Report on Test Data (SMOTE):")
print(classification_report(y_test, y_test_pred_smote))

### 방법 3: 언더샘플링 ###
print("\n=== Logistic Regression with Undersampling ===")
undersampler = RandomUnderSampler(random_state=42)
X_train_under, y_train_under = undersampler.fit_resample(X_train, y_train)

print("Resampled Training Set Size (Undersampling):", X_train_under.shape)

model_under = LogisticRegression(random_state=42, max_iter=1000)
model_under.fit(X_train_under, y_train_under)

# 테스트 데이터 평가
y_test_pred_under = model_under.predict(X_test)
print("\nClassification Report on Test Data (Undersampling):")
print(classification_report(y_test, y_test_pred_under))


=== Logistic Regression with Class Weights ===

Classification Report on Test Data (Class Weights):
              precision    recall  f1-score   support

           0       0.25      0.65      0.37       450
           1       0.93      0.70      0.80      2850

    accuracy                           0.69      3300
   macro avg       0.59      0.67      0.58      3300
weighted avg       0.84      0.69      0.74      3300


=== Logistic Regression with SMOTE ===
Resampled Training Set Size (SMOTE): (26600, 15)

Classification Report on Test Data (SMOTE):
              precision    recall  f1-score   support

           0       0.25      0.65      0.37       450
           1       0.93      0.70      0.80      2850

    accuracy                           0.69      3300
   macro avg       0.59      0.68      0.58      3300
weighted avg       0.84      0.69      0.74      3300


=== Logistic Regression with Undersampling ===
Resampled Training Set Size (Undersampling): (4200, 15)

Classif